In [5]:
settings = {
    'URL': 'https://tmtgroup.sharepoint.com/sites/ABMDataPlatform',
    'USERNAME': 'yusuf.badurohman@mst.co.id',
    'PASSWORD': 'MSTP@ssw0rd',
    'LIST': 'TR_TRIP',
    'S3_BUCKET': 'abm-data-platform-s3-raw'
}

In [6]:
import json
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.runtime.utilities.request_options import RequestOptions
from io import StringIO
from datetime import datetime
import pandas as pd
import boto3

ctxAuth = AuthenticationContext(url=settings['URL'])
if ctxAuth.acquire_token_for_user(username=settings['USERNAME'], password=settings['PASSWORD']):
    ctx = ClientContext(settings['URL'], ctxAuth)

In [19]:
options = RequestOptions("{0}/_api/web/".format(settings['URL']))
options.set_header('Accept', 'application/json')
options.set_header('Content-Type', 'application/json')
data = ctx.execute_request_direct(options)
s = json.loads(data.content)
print(s)

{'odata.metadata': 'https://tmtgroup.sharepoint.com/sites/ABMDataPlatform/_api/$metadata#SP.ApiData.Webs/@Element', 'odata.type': 'SP.Web', 'odata.id': 'https://tmtgroup.sharepoint.com/sites/ABMDataPlatform/_api/Web', 'odata.editLink': 'Web', 'AllowRssFeeds': True, 'AlternateCssUrl': '', 'AppInstanceId': '00000000-0000-0000-0000-000000000000', 'ClassicWelcomePage': None, 'Configuration': 0, 'Created': '2019-08-16T04:07:26.817', 'CurrentChangeToken': {'StringValue': '1;2;47eb1508-c071-4d74-8c0b-a8d81bffec7e;637024015838700000;236845792'}, 'CustomMasterUrl': '/sites/ABMDataPlatform/_catalogs/masterpage/seattle.master', 'Description': '', 'DesignPackageId': '00000000-0000-0000-0000-000000000000', 'DocumentLibraryCalloutOfficeWebAppPreviewersDisabled': False, 'EnableMinimalDownload': False, 'FooterEmphasis': 0, 'FooterEnabled': False, 'FooterLayout': 0, 'HeaderEmphasis': 0, 'HeaderLayout': 0, 'HorizontalQuickLaunch': False, 'Id': '47eb1508-c071-4d74-8c0b-a8d81bffec7e', 'IsHomepageModernize

In [44]:
# Get fields from List dynamically

fieldId = []
fieldTitle = []
options = RequestOptions("{0}/_api/lists/getbytitle('TR_TRIP')/fields?$filter=Hidden eq false and ReadOnlyField eq false and Group eq 'Custom Columns'".format(settings['URL']))
options.set_header('Accept', 'application/json')
options.set_header('Content-Type', 'application/json')
data = ctx.execute_request_direct(options)
fields = json.loads(data.content)
print('TOTAL FIELDS: ', len(fields['value']))
for idx, field in enumerate(fields['value']):
    fieldId.append(field['EntityPropertyName'])
    fieldTitle.append(field['Title'])
    print("{2}. ID: {0}, TITLE: {1}".format(field['EntityPropertyName'], field['Title'], idx + 1))

TOTAL FIELDS:  77
1. ID: Title, TITLE: ID_MS_FLEET_TR
2. ID: ID_MS_BARGE, TITLE: ID_MS_BARGE
3. ID: FLEET_SPOT_NAME, TITLE: FLEET_SPOT_NAME
4. ID: SPOT_TUG, TITLE: SPOT_TUG
5. ID: SPOT_BARGE, TITLE: SPOT_BARGE
6. ID: ID_MS_CUST_TR, TITLE: ID_MS_CUST_TR
7. ID: ID_MS_SERVICE_TR, TITLE: ID_MS_SERVICE_TR
8. ID: CARGO_NUM, TITLE: CARGO_NUM
9. ID: TRV_NUM, TITLE: TRV_NUM
10. ID: MV_NAME_LOOK, TITLE: MV_NAME_LOOK
11. ID: DATA_OWNER, TITLE: DATA_OWNER
12. ID: DATA_DATE, TITLE: DATA_DATE
13. ID: POL_TXT, TITLE: POL_TXT
14. ID: POD_TXT, TITLE: POD_TXT
15. ID: VESS_POL_ARR, TITLE: VESS_POL_ARR
16. ID: VESS_POL_ASIDE, TITLE: VESS_POL_ASIDE
17. ID: VESS_POL_COMM_LOAD, TITLE: VESS_POL_COMM_LOAD
18. ID: VESS_POL_COMP_LOAD, TITLE: VESS_POL_COMP_LOAD
19. ID: VESS_POL_COFF, TITLE: VESS_POL_COFF
20. ID: DOC_POL_OVERHD, TITLE: DOC_POL_OVERHD
21. ID: DOC_POL_ONB, TITLE: DOC_POL_ONB
22. ID: VESS_POL_DEP, TITLE: VESS_POL_DEP
23. ID: VESS_POD_ARR, TITLE: VESS_POD_ARR
24. ID: VESS_POD_ASIDE, TITLE: VESS_POD_AS

In [45]:
print(fieldId)

['Title', 'ID_MS_BARGE', 'FLEET_SPOT_NAME', 'SPOT_TUG', 'SPOT_BARGE', 'ID_MS_CUST_TR', 'ID_MS_SERVICE_TR', 'CARGO_NUM', 'TRV_NUM', 'MV_NAME_LOOK', 'DATA_OWNER', 'DATA_DATE', 'POL_TXT', 'POD_TXT', 'VESS_POL_ARR', 'VESS_POL_ASIDE', 'VESS_POL_COMM_LOAD', 'VESS_POL_COMP_LOAD', 'VESS_POL_COFF', 'DOC_POL_OVERHD', 'DOC_POL_ONB', 'VESS_POL_DEP', 'VESS_POD_ARR', 'VESS_POD_ASIDE', 'VESS_POD_COMM_LOAD', 'VESS_POD_COMP_LOAD', 'VESS_POD_COFF', 'VESS_POD_DEP', 'VESS_POL_ARR_2', 'ASSIST_POL_IN', 'ASSIST_POL_OUT', 'ASSIST_POD_IN', 'ASSIST_POD_OUT', 'BUNKER_ROB_2', 'BUNKER_ROB_1', 'COST_AGENCY_NUM', 'COST_DUES_NUM', 'REMARKS_TXT', 'BONUS_TRNSHP_NUM', 'ID_MS_FLEET_TR_x003a_ID_MS_FLEET', 'ID_MS_CUST_TR_x003a_ID_MS_CUST', 'ID_MS_SERVICE_TR_x003a_ID_MS_SER', 'STAT_TXT', 'TOTAL_BUNKER_CONS', 'SAVING_BUNKER', 'VOYAGE_CODE', 'ID_MS_BARGE_x003a_ID_MS_BARGE', 'CHECK_DATEFILL', 'MONTH_NAME', 'YEAR_NAME', 'FLEET_ACTIVITY_STATUS', 'FLEET_LOCATION', 'FLEET_ACTIVITY_DATE', 'STATUS_RC', 'RC_CARGO_VOLUME', 'MV_COMPLET

In [65]:
print(fieldTitle)

['ID_MS_FLEET_TR', 'ID_MS_BARGE', 'FLEET_SPOT_NAME', 'SPOT_TUG', 'SPOT_BARGE', 'ID_MS_CUST_TR', 'ID_MS_SERVICE_TR', 'CARGO_NUM', 'TRV_NUM', 'MV_NAME_LOOK', 'DATA_OWNER', 'DATA_DATE', 'POL_TXT', 'POD_TXT', 'VESS_POL_ARR', 'VESS_POL_ASIDE', 'VESS_POL_COMM_LOAD', 'VESS_POL_COMP_LOAD', 'VESS_POL_COFF', 'DOC_POL_OVERHD', 'DOC_POL_ONB', 'VESS_POL_DEP', 'VESS_POD_ARR', 'VESS_POD_ASIDE', 'VESS_POD_COMM_LOAD', 'VESS_POD_COMP_LOAD', 'VESS_POD_COFF', 'VESS_POD_DEP', 'VESS_POL_ARR_2', 'ASSIST_POL_IN', 'ASSIST_POL_OUT', 'ASSIST_POD_IN', 'ASSIST_POD_OUT', 'BUNKER_ROB_2', 'BUNKER_ROB_1', 'COST_AGENCY_NUM', 'COST_DUES_NUM', 'REMARKS_TXT', 'BONUS_TRNSHP_NUM', 'ID_MS_FLEET_TR:ID_MS_FLEET', 'ID_MS_CUST_TR:ID_MS_CUST', 'ID_MS_SERVICE_TR:ID_MS_SERVICE', 'STAT_TXT', 'TOTAL_BUNKER_CONS', 'SAVING_BUNKER', 'VOYAGE_CODE', 'ID_MS_BARGE:ID_MS_BARGE', 'CHECK_DATEFILL', 'MONTH_NAME', 'YEAR_NAME', 'FLEET_ACTIVITY_STATUS', 'FLEET_LOCATION', 'FLEET_ACTIVITY_DATE', 'STATUS_RC', 'RC_CARGO_VOLUME', 'MV_COMPLETION', 'MV_N

In [53]:
# Get next URL

options = RequestOptions("{0}/_api/lists/getbytitle('TR_TRIP')/items?$top=5".format(settings['URL']))
options.set_header('Accept', 'application/json')
options.set_header('Content-Type', 'application/json')
data = ctx.execute_request_direct(options)
items = json.loads(data.content)
list_items = []
for item in items['value']:
    list_item = []
    for field in fieldId:
        list_item.append(item[field])
    list_items.append(list_item)
print(list_items)


[['KAILI II', 'MOANA II', None, None, None, 'TIA', 'TRANSHIPMENT', 1957.541, 7.0, 'MV. ADRIATICA GRAECA', 'Trophy Geo Disansyah', '2019-01-07T23:00:00Z', 'Bunati Anchorage', 'Bunati Anchorage', None, None, None, None, None, None, None, None, '2018-12-31T19:00:00Z', '2018-12-31T20:40:00Z', '2018-12-31T21:40:00Z', '2019-01-01T01:25:00Z', '2019-01-01T01:35:00Z', '2019-01-01T01:50:00Z', '2019-01-01T23:00:00Z', None, None, 'KAILI I', 'TRISTAN SATU', 19458.0, 21024.0, None, None, 'Rc from mv aifanourios', None, '2.00000000000000', '1.00000000000000', '1.00000000000000', None, 1566.0, 1134.0, None, '2.00000000000000', 1.0, 'JAN', '2019', 'Full Away to Bunati', 'N/A', '22-01-2019 00:15', True, 1957.541, True, '209', 'SITE BUNATI', None, '2018-12-21 15:00:00', None, '69728.2840000000', None, None, None, None, '72800.0000000000', None, 1.0, '2019-01-22T23:15:39Z', 1957.541, 'JAN', '2019', '2019-01-22T23:15:39Z', 'Arya Wirawan', None, False], ['TRISTAN SATU', 'AQUILA SATU', None, None, None, 'TIA

In [13]:
# Get data from List

list_object = ctx.web.lists.get_by_title(settings['LIST'])
items = list_object.get_items()
ctx.load(items)
ctx.execute_query()
list_items = []
for item in items:
    list_item = []
    for field in fieldId:
        list_item.append(item.properties[field])
    list_items.append(list_item)
df = pd.DataFrame(list_items, columns=fieldTitle)   
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)
print('TOTAL DATA: ', len(items))
print(df)

TOTAL DATA:  100
   ID_MS_FLEET_TR  ID_MS_BARGE                    FLEET_SPOT_NAME  \
0        KAILI II     MOANA II                               None   
1    TRISTAN SATU  AQUILA SATU                               None   
2       KAILI III    MOANA III                               None   
3         KAILI V      MOANA V                               None   
4       KAILI VII    MOANA VII                               None   
..            ...          ...                                ...   
95       KAILI II     MOANA II                               None   
96           None         None  TB. MICHELLE VII / BG. BENAMI VII   
97        KAILI V      MOANA V                               None   
98   TRISTAN SATU  AQUILA SATU                               None   
99        KAILI I      MOANA I                               None   

            SPOT_TUG      SPOT_BARGE               ID_MS_CUST_TR  \
0               None            None                         TIA   
1               No

In [21]:
now = datetime.now()
datetime_string = now.strftime("%Y%m%d%H%M%S")
object_url = 'bdd/raw/TR_TRIP_{0}.csv'.format(datetime_string)

s3_resource = boto3.resource('s3')
s3_resource.Object(settings['S3_BUCKET'], object_url).put(Body=csv_buffer.getvalue())
print('Object URL: ', object_url)

Object URL:  bdd/raw/TR_TRIP_20190823140504.csv
